## Preprocessing

In [1]:
DATASETS = ["ibtracs.last3years"]

In [2]:
from src.preprocessing.load_data import load_dataset
from src.preprocessing.pipeline import preprocess

In [3]:
for ds in DATASETS:
    df_raw, cfg = load_dataset(ds)
    X, y = preprocess(df_raw.copy(), cfg)

/Users/anitarazafi/Desktop/masters/code/feature-selection/src/preprocessing/clean.py:67: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(df[col], format="%Y-%m-%d %H:%M:%S", infer_datetime_format=True, utc=True)


ValueError: time data " " doesn't match format "%Y-%m-%d %H:%M:%S", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
df_raw.shape, X.shape